In [21]:
from sqlalchemy import create_engine, text
import pandas as pd

# Parámetros de conexión
user = 'jbpm'
password = 'jbpm'
host = 'localhost'
port = '5432'
database = 'jbpm'

# Crea la URL de conexión
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# Probar conexión: listar tablas
with engine.connect() as connection:
    result = connection.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public'"))
    for row in result:
        # Imprimir usando el índice de la tupla
        print(row[0])  # row[0] es el valor de la primera columna (table_name)



booleanexpression
audittaskimpl
bamtasksummary
correlationkeyinfo
casefiledatalog
caseidinfo
caseroleassignmentlog
content
contextmappinginfo
deadline
attachment
delegation_delegates
deploymentstore
eventtypes
errorinfo
email_header
executionerrorinfo
i18ntext
nodeinstancelog
processinstancelog
querydefinitionstore
organizationalentity
processinstanceinfo
reassignment
notification
notification_bas
notification_email_header
notification_recipients
peopleassignments_bas
peopleassignments_exclowners
peopleassignments_potowners
peopleassignments_recipients
peopleassignments_stakeholders
reassignment_potentialowners
sessioninfo
requestinfo
taskdef
taskevent
taskvariableimpl
variableinstancelog
workiteminfo
task_comment
task
escalation
correlationpropertyinfo


In [2]:
df = pd.read_sql("select name, processinstanceid, value from taskvariableimpl t ;", engine)
df.head()


,name,processinstanceid,value
0,Nombre,1,Francisco de Jesús Contreras Loo
1,archivoAdjuntoD,1,prueba.pdf (1).pdf####24043####2025-04-29 19:5...
2,Motivo,1,Esta es una prueba final


In [3]:
df_pivot = df.pivot(index='processinstanceid', columns='name', values='value').reset_index()
df_pivot.columns.name = None
df_pivot.head()


,processinstanceid,Motivo,Nombre,archivoAdjuntoD
0,1,Esta es una prueba final,Francisco de Jesús Contreras Loo,prueba.pdf (1).pdf####24043####2025-04-29 19:5...


In [4]:
df_pivot.to_sql(
    name='x_tabla_prueba',    # Nombre de la tabla en PostgreSQL
    con=engine,              # Conexión a la base de datos
    if_exists='append',      # Crea la tabla o la reemplaza si ya existe
    index=False              # No incluir el índice de pandas como columna
)

1

In [5]:
df_verificacion = pd.read_sql('SELECT * FROM "public"."x_tabla_prueba"', engine)
df_verificacion.tail()

,processinstanceid,Motivo,Nombre,archivoAdjuntoD
0,1,Esta es una prueba final,Francisco de Jesús Contreras Loo,prueba.pdf (1).pdf####24043####2025-04-29 19:5...
1,1,Esta es una prueba final,Francisco de Jesús Contreras Loo,prueba.pdf (1).pdf####24043####2025-04-29 19:5...


# UNION DE TABLAS POR ID, NOMBRE, INSTANCIA, TAREA, ETC

In [22]:
# TABLA taskvariableimpl

df_vars = pd.read_sql("""
    SELECT processinstanceid, taskid, name as Campo_llenado, value as registro_de_campo
    FROM taskvariableimpl
""", engine)
df_vars.tail()


,processinstanceid,taskid,campo_llenado,registro_de_campo
0,1,1,archivoAdjuntoUno,Guia_Persistencia_jBPM (1) (1) (2) (1).pdf####...
1,1,1,nombre,Jack Contreras Loo
2,1,2,archivoAdjuntoDos,Guia_Persistencia_jBPM (1) (1) (2) (1) (1).pdf...
3,1,2,fecha_de_inicio,Wed Apr 30 18:55:00 UTC 2025
4,1,3,motivo_de_,Este es un motivo de prueba para John


In [23]:
# TABLA Tareas
df_tasks = pd.read_sql("""
    SELECT processinstanceid, id AS taskid, actualowner_id, formname as nombre_de_formulario
    FROM task
""", engine)
df_tasks.tail()

,processinstanceid,taskid,actualowner_id,nombre_de_formulario
0,1,1,jack,Tarea1_de_Jack
1,1,2,katy,Tarea2_de_Katy
2,1,3,john,Tarea3_de_John


In [24]:
# Paso 3: Unir variables con tareas (para saber quién hizo qué)
df_combined = pd.merge(df_tasks, df_vars, on=['processinstanceid', 'taskid'], how='left')
df_combined.tail()

,processinstanceid,taskid,actualowner_id,nombre_de_formulario,campo_llenado,registro_de_campo
0,1,1,jack,Tarea1_de_Jack,archivoAdjuntoUno,Guia_Persistencia_jBPM (1) (1) (2) (1).pdf####...
1,1,1,jack,Tarea1_de_Jack,nombre,Jack Contreras Loo
2,1,2,katy,Tarea2_de_Katy,archivoAdjuntoDos,Guia_Persistencia_jBPM (1) (1) (2) (1) (1).pdf...
3,1,2,katy,Tarea2_de_Katy,fecha_de_inicio,Wed Apr 30 18:55:00 UTC 2025
4,1,3,john,Tarea3_de_John,motivo_de_,Este es un motivo de prueba para John
